In [1]:
from keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout

def to_profile():
    Conv2D(32, kernel_size=3, activation='relu', input_shape=(28, 28, 1), kernel_initializer='he_normal', name='Conv2D-1')
    MaxPooling2D(pool_size=2, name='MaxPool')
    Dropout(0.25, name='Dropout-1')
    Conv2D(64, kernel_size=3, activation='relu', name='Conv2D-2')
    Dropout(0.25, name='Dropout-2')
    Conv2D(128, kernel_size=3, activation='relu', name='Conv2D-3')
    Dropout(0.4, name='Dropout-3')
    Flatten(name='flatten')
    Dense(128, activation='relu', name='Dense')
    Dropout(0.4, name='Dropout')
    Dense(10, activation='softmax', name='Output')

2023-02-11 01:25:59.935403: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-11 01:26:00.151161: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-02-11 01:26:01.140870: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/include:/usr/local/cuda/lib64::/usr/local/cuda/extras/CUPTI/lib64
2023-

In [2]:
%load_ext line_profiler

In [3]:
image_shape = (28, 28, 1)
%lprun -u 1e-06 -f to_profile to_profile()

Timer unit: 1e-06 s

Total time: 0.0343031 s
File: /tmp/ipykernel_1339/2063558768.py
Function: to_profile at line 3

Line #      Hits         Time  Per Hit   % Time  Line Contents
     3                                           def to_profile():
     4         1      20528.4  20528.4     59.8      Conv2D(32, kernel_size=3, activation='relu', input_shape=(28, 28, 1), kernel_initializer='he_normal', name='Conv2D-1')
     5         1        982.6    982.6      2.9      MaxPooling2D(pool_size=2, name='MaxPool')
     6         1       1400.7   1400.7      4.1      Dropout(0.25, name='Dropout-1')
     7         1       2695.2   2695.2      7.9      Conv2D(64, kernel_size=3, activation='relu', name='Conv2D-2')
     8         1        674.9    674.9      2.0      Dropout(0.25, name='Dropout-2')
     9         1       1342.9   1342.9      3.9      Conv2D(128, kernel_size=3, activation='relu', name='Conv2D-3')
    10         1        876.7    876.7      2.6      Dropout(0.4, name='Dropout-3')
 

In [4]:
import tracemalloc

def mem_profile():
    tracemalloc.start()
    _ = Conv2D(32, kernel_size=3, activation='relu', input_shape=(28, 28, 1), kernel_initializer='he_normal', name='Conv2D-1')
    base, peak = tracemalloc.get_traced_memory()
    base /= 1024 * 2
    peak /= 1024 * 2
    print(f'Conv2D: base: {base:.2f}MiB, peak: {peak:.2f}MiB, increment: {peak - base:.2f}MiB')
    tracemalloc.clear_traces()

    _ = MaxPooling2D(pool_size=2, name='MaxPool')
    base, peak = tracemalloc.get_traced_memory()
    base /= 1024 * 2
    peak /= 1024 * 2
    print(f'MaxPool: base: {base:.2f}MiB, peak: {peak:.2f}MiB, increment: {peak - base:.2f}MiB')
    tracemalloc.clear_traces()

    _ = Dropout(0.25, name='Dropout-1')
    base, peak = tracemalloc.get_traced_memory()
    base /= 1024 * 2
    peak /= 1024 * 2
    print(f'Dropout: base: {base:.2f}MiB, peak: {peak:.2f}MiB, increment: {peak - base:.2f}MiB')
    tracemalloc.clear_traces()

    _ = Conv2D(64, kernel_size=3, activation='relu', name='Conv2D-2')
    base, peak = tracemalloc.get_traced_memory()
    base /= 1024 * 2
    peak /= 1024 * 2
    print(f'Conv2D: base: {base:.2f}MiB, peak: {peak:.2f}MiB, increment: {peak - base:.2f}MiB')
    tracemalloc.clear_traces()
    
    _ = Dropout(0.25, name='Dropout-2')
    base, peak = tracemalloc.get_traced_memory()
    base /= 1024 * 2
    peak /= 1024 * 2
    print(f'Dropout: base: {base:.2f}MiB, peak: {peak:.2f}MiB, increment: {peak - base:.2f}MiB')
    tracemalloc.clear_traces()
    
    _ = Conv2D(128, kernel_size=3, activation='relu', name='Conv2D-3')
    base, peak = tracemalloc.get_traced_memory()
    base /= 1024 * 2
    peak /= 1024 * 2
    print(f'Conv2D: base: {base:.2f}MiB, peak: {peak:.2f}MiB, increment: {peak - base:.2f}MiB')
    tracemalloc.clear_traces()
    
    _ = Dropout(0.4, name='Dropout-3')
    base, peak = tracemalloc.get_traced_memory()
    base /= 1024 * 2
    peak /= 1024 * 2
    print(f'Dropout: base: {base:.2f}MiB, peak: {peak:.2f}MiB, increment: {peak - base:.2f}MiB')
    tracemalloc.clear_traces()

    _ = Flatten(name='flatten')
    base, peak = tracemalloc.get_traced_memory()
    base /= 1024 * 2
    peak /= 1024 * 2
    print(f'Flatten: base: {base:.2f}MiB, peak: {peak:.2f}MiB, increment: {peak - base:.2f}MiB')
    tracemalloc.clear_traces()

    _ = Dense(128, activation='relu', name='Dense')
    base, peak = tracemalloc.get_traced_memory()
    base /= 1024 * 2
    peak /= 1024 * 2
    print(f'Dense: base: {base:.2f}MiB, peak: {peak:.2f}MiB, increment: {peak - base:.2f}MiB')
    tracemalloc.clear_traces()

    _ = Dropout(0.4, name='Dropout')
    base, peak = tracemalloc.get_traced_memory()
    base /= 1024 * 2
    peak /= 1024 * 2
    print(f'Dropout: base: {base:.2f}MiB, peak: {peak:.2f}MiB, increment: {peak - base:.2f}MiB')
    tracemalloc.clear_traces()
    
    _ = Dense(10, activation='softmax', name='Output')
    base, peak = tracemalloc.get_traced_memory()
    base /= 1024 * 2
    peak /= 1024 * 2
    print(f'Dense: base: {base:.2f}MiB, peak: {peak:.2f}MiB, increment: {peak - base:.2f}MiB')
    tracemalloc.stop()

mem_profile()

Conv2D: base: 3.97MiB, peak: 5.94MiB, increment: 1.96MiB
MaxPool: base: 2.96MiB, peak: 3.55MiB, increment: 0.60MiB
Dropout: base: 2.77MiB, peak: 3.41MiB, increment: 0.64MiB
Conv2D: base: 3.22MiB, peak: 5.18MiB, increment: 1.96MiB
Dropout: base: 4.75MiB, peak: 5.42MiB, increment: 0.67MiB
Conv2D: base: 2.91MiB, peak: 4.87MiB, increment: 1.96MiB
Dropout: base: 2.52MiB, peak: 3.19MiB, increment: 0.67MiB
Flatten: base: 1.70MiB, peak: 2.73MiB, increment: 1.03MiB
Dense: base: 2.53MiB, peak: 4.17MiB, increment: 1.64MiB
Dropout: base: 2.25MiB, peak: 2.92MiB, increment: 0.67MiB
Dense: base: 2.84MiB, peak: 4.48MiB, increment: 1.64MiB
